In [3]:
!pipenv install datasets

Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Installing datasets...
Resolving datasets...
Adding datasets to Pipfile's [packages] ...
✔ Installation Succeededts...
⠋ Installing datasets...
Pipfile.lock (3381d9) out of date, updating to (088ea1)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠧ Locking...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (28dca4bfd152f26865c387643ac3e07fb2bb0ce28740cfa38c84449fc4088ea1)!
Installing dependencies from Pipfile.lock (088ea1)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [92]:
from transformers import AutoModel, AutoModelForSeq2SeqLM, DistilBertForQuestionAnswering, DistilBertForSequenceClassification
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType, PeftConfig
import torch
from datasets import load_dataset
import os
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from tqdm import tqdm
from datasets import load_dataset

device = "mps"
model_name_or_path = "sentence-transformers/distiluse-base-multilingual-cased-v1"
tokenizer_name_or_path = "sentence-transformers/distiluse-base-multilingual-cased-v1"
peft_config = PromptTuningConfig(
    task_type=TaskType.QUESTION_ANS,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text="Find related documents: ",
    tokenizer_name_or_path=model_name_or_path,
    num_layers=6,
    token_dim=768,
    num_attention_heads=12
)

checkpoint_name = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/", "_"
)
max_length = 64
lr = 3e-2
num_epochs = 50
batch_size = 8

In [44]:
from datasets import load_dataset

dataset = load_dataset("harheem/deco_qa", 'default')

print(dataset)
dataset["train"][0]

DatasetDict({
    train: Dataset({
        features: ['query', 'answer'],
        num_rows: 5152
    })
})


{'query': '면진장치가 뭐야?',
 'answer': '면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. 주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다.'}

In [46]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [70]:
def preprocess_function(examples):
    inputs = [f"Query: {x}" for x in examples['query']]
    targets = examples['answer']
    model_inputs = tokenizer(inputs, padding='max_length', truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, padding='max_length', truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset['train'].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

train_dataset = processed_datasets['train']
eval_dataset = processed_datasets["train"]

Running tokenizer on dataset: 100%|██████████| 5152/5152 [00:00<00:00, 6201.62 examples/s]


In [71]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size)

In [93]:
# creating model
model = DistilBertForSequenceClassification.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/distiluse-base-multilingual-cased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 6,144 || all params: 135,332,354 || trainable%: 0.004539934330854837


In [90]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [94]:
# training and evaluation
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    # Evaluation loop
    model.eval()
    eval_loss = 0
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]
            loss = outputs.loss
            eval_loss += loss.item()

    # Calculate and print losses and perplexities
    eval_epoch_loss = eval_loss / len(eval_dataloader)
    train_epoch_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch}: Train Loss: {train_epoch_loss}, Eval Loss: {eval_epoch_loss}")

  0%|          | 0/644 [00:00<?, ?it/s]


TypeError: DistilBertForSequenceClassification.forward() got an unexpected keyword argument 'start_positions'

In [126]:
import pandas as pd

train = pd.read_csv("train.csv")

In [127]:
train['query']

0                                   면진장치가 뭐야?
1                              내진설계의 종류 좀 알려줘
2                               철골구조의 장점이 뭐야?
3                           철골철근 콘크리트 구조가 뭐야?
4                            철골구조는 어떤 방식이 있어?
                        ...                  
5147             벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?
5148         벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?
5149    외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?
5150        창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?
5151      AD, PD에 면한 벽체 결로에 대한 대책은 어떤 것이 있나요?
Name: query, Length: 5152, dtype: object

In [128]:
train['answer']

0       면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...
1       내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...
2       철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...
3       철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...
4       철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...
                              ...                        
5147    벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
5148    AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각...
5149    외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
5150    창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및...
5151    AD, PD에 면한 벽체의 결로에 대한 대책으로는 단열재를 미실하게 시공하여 결로가...
Name: answer, Length: 5152, dtype: object

In [132]:
from sentence_transformers import SentenceTransformer
import scipy.spatial
import numpy as np

# 모델 로드
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
soft_prompt_embedding = np.zeros(10, dtype=float)

# 문장 임베딩 생성
query_embedding = model.encode(train['query'].iloc[0])
answer_embedding = model.encode(train['answer'].iloc[0])

retrieval_embeddings = np.concatenate([soft_prompt_embedding, query_embedding[10:]])

cosine_similarity = 1 - scipy.spatial.distance.cosine(retrieval_embeddings, answer_embedding)

# print("코사인 유사도:", cosine_similarity)


In [133]:
retrieval_embeddings.shape

(512,)

In [120]:
embeddings[0][10:]

502

In [117]:
type(embeddings[0])

numpy.ndarray